## Exercise 4
### By: Alexis Morse

### 1. You will use the wiki_edit.txt file from Exercise 1. Make sure the data file is in the directory where you will be writing your python code.

In [ ]:
import pandas as p
column_names = ['RevisionId', 'ArticleName', 'Timestamp', 'Editor']
data = p.read_table('wiki_edit.txt',sep=' ',header=None,names=column_names)
data[:4]

### 2. Write a python script using the mysql-connector library to performs the following steps:

##### a. Open a connection to the CSE database server (mysql-user-cse.egr.msu.edu). Your username and database name is the same as your MSU NetID. If you’re using the database for the first time, the password is your student PID. Note: if you can’t access the database server from outside the campus, you may need to use the EGR VPN or do the exercise on campus. Please refer to our announcement on d2l (on Sep 19) for further information.
##### b. Create a table named Wiki_Edit, which contains the following schema: 
        RevisionID: int Primary Key
        ArticleName: varchar(100) EditDate: date
        UserName: varchar(50)
##### c. Read the file wiki_edit.txt and parse each line to identify the individual fields. For this exercise, do not use the read_table and DataFrame approach shown in lecture 4. Instead, you should read the file directly, parse each line, and store the fields from each line into the database. You should refer to the example given in slide #16 in lecture 2 on how to read from the data file directly and split each line into the individual fields.
##### d. Query the database to find the Article name with largest number of edits.
##### e. NOTE: you can only connect to the CSE database server if you’re on the EGR network. Try todo the exercise on campus. Otherwise, you need to install mysql on your own laptop.

In [ ]:
pip install mysql.connector

In [ ]:
import mysql.connector
from mysql.connector import errorcode

In [ ]:
uname = "morseal3"
pwd = "jNlNs2023"
hname = "mysql-user-cse.egr.msu.edu"

In [ ]:
### Question (a) & (b)

import mysql.connector
from mysql.connector import errorcode

try:
    cnx = mysql.connector.connect(user=uname, password=pwd, host=hname, database=uname)
    cursor = cnx.cursor()
    query = "DROP TABLE IF EXISTS Wiki_Edit"
    cursor.execute(query)

    query = """CREATE TABLE IF NOT EXISTS Wiki_Edit (RevId int PRIMARY KEY,
                                ArticleName varchar(100), EditDate date, User varchar(50))
            """
    cursor.execute(query)
    cursor.close()
    cnx.commit()
    cnx.close()
    
except:
    print(mysql.connector.Error)


In [ ]:
### Question (c)
def byUnicode(char):
    if ord(char) < 48 or ord(char) > 127: return ''
    else: return char

try:
    cnx = mysql.connector.connect(user=uname, password=pwd, host=hname, database=uname)
    cursor = cnx.cursor()
    
    for index, row in data.iterrows():
        rid = row['RevisionId']
        Article = ''.join(list(filter(byUnicode, row['ArticleName'])))
        EditDate = row['Timestamp']
        User = row['Editor']
        if index < 100:
            query = "INSERT INTO Wiki_Edit VALUES (" + str(rid) + ",'" 
            query += Article + "','" + EditDate + "','"+ User + "')"""
            cursor.execute(query)
        
    cnx.commit()
    cursor.close()
    cnx.close()
    
except:
    print(mysql.connector.Error)   

In [ ]:
### Question (d)
try:
    cnx = mysql.connector.connect(user=uname, password=pwd, host=hname, database=uname)
    cursor = cnx.cursor()
    query = "SELECT ArticleName, COUNT(ArticleName) as occur FROM Wiki_Edit GROUP BY ArticleName ORDER BY occur DESC LIMIT 1;"
    cursor.execute(query)
    
    for (rid, article, edate, user) in cursor:
        print(article)
        

    cursor.close()
    cnx.close()
    
except:
    print(mysql.connector.Error) 
    


### 3. Install MongoDB to your machine.
##### a. Launch the MongoDB server.
##### b. Using pymongo, write an Jupyter notebook script that will store the entries in the wiki_edit.txt file to a collection named wiki in MongoDB. To store the data, you need to convert each line into a dictionary object. The keys to the dictionary correspond to RevisionID, ArticleName, EditDate, and UserName and the values correspond to the individual entries in each line of the file:
        columns = line.strip().split(‘ ‘)
        record = { ‘RevisionID’: columns[0], ...., ‘UserName’: columns[3]} db.wiki.insert_one(record)
##### c. Write a query to find the date and username of editors who edited the “Anarchism” article: 
    revisions = db.wiki 
    for line in revisions.find({"ArticleName": "Anarchism"}): print line["EditDate"], line["UserName"]

In [ ]:
pip install pymongo

In [ ]:
### Question (a)

from pymongo import MongoClient
client = pymongo.MongoClient('localhost', 27017)
db = client['Wiki']
collection = db['entries']

In [ ]:
### Question (b)

for line in data:
    columns = line.strip().split('')
    record = {'RevisionID': columns[0], 'ArticleName': columns[1], 'EditDate': columns[2], 'UserName': columns[3]} 
    db.wiki.insert_one(record)

db.createCollection(wiki, collection)

In [ ]:
### Question (c)
revisions = db.wiki 
for line in revisions:
    if line.key() == "ArticleName":
        print(line["EditDate"], line["UserName"])
    if line.key() == "Anarchism":
        print(line["EditDate"], line["UserName"])